<a href="https://www.kaggle.com/code/lyxbash/vgg16-potato-classification?scriptVersionId=133763178" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Input layer is used to define the shape of the input data for the model.
#Lambda layer is used to define custom operations or functions that transform the input data.
#Dense layer represents a fully connected layer, where each neuron is connected to every neuron in the previous layer.
#Flatten layer is used to flatten the input data into a 1-dimensional array.
from keras.layers import Input, Lambda, Dense, Flatten


#The Model class is used to create the model architecture by specifying the input and output layers.
from keras.models import Model


#This line imports the pre-trained VGG16 model from Keras. 
# VGG16 is a convolutional neural network architecture that has been pre-trained on a large dataset called ImageNet. 
# Importing it allows you to use the pre-trained weights and architecture for your own tasks.
from keras.applications.vgg16 import VGG16


# The preprocess_input function is used to preprocess input images according to the requirements of the VGG16 model. 
# It performs necessary preprocessing steps such as mean subtraction and 
# scaling to prepare the image data for input to the model.
from keras.applications.vgg16 import preprocess_input


# The image module provides functions for loading and manipulating images.
# It is commonly used for loading images into NumPy arrays and performing various image operations.
from keras.preprocessing import image


# The ImageDataGenerator class is used for data augmentation and generating batches of image data during training. 
# It provides various methods to perform data augmentation techniques such as rotation, scaling, 
# and flipping, which helps to increase the diversity of the training data.
from keras.preprocessing.image import ImageDataGenerator


# The Sequential class is an alternative way to define a model architecture in Keras. 
# It allows you to build a model by stacking layers sequentially, one on top of the other.
from keras.models import Sequential


# The Input layer is used to define the input shape of the model. 
#It specifies the shape of the input data, such as the dimensions of an image or the length of a sequence.
from keras.layers import Input


#he glob function is used to retrieve filenames or pathnames that match a specific pattern.
#It is often used for file handling and directory operations.
from glob import glob

import matplotlib.pyplot as plt
import numpy as np

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [2]:
input_shape = (224, 224, 3)

In [3]:
#the model will be initialized with pre-trained weights obtained by training on the ImageNet dataset.

#The include_top parameter is set to False, which means that the final fully connected layers (classification layers) 
#of the VGG16 model will not be included. By excluding the top layers, we can add our own custom layers for our specific task.
vgg = VGG16(input_shape = input_shape, weights = 'imagenet', include_top = False)
vgg.summary()


# trainable attribute of each layer to False. By doing this, we freeze the weights of all the layers in the VGG16 model, 
#preventing them from being updated during training.
for layer in vgg.layers:
    layer.trainable = True

58889256/58889256 [==============================] - 0s 0us/step
Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                              

In [4]:
#glob module is used to retrieve subdirectories within the 'train/' directory.
folders = glob('/kaggle/input/potato-disease/train/*')

In [5]:
#The Flatten layer takes the multidimensional output from the previous layer (which is the output of the VGG16 model) and 
#converts it into a one-dimensional array. This flattening operation is necessary to 
#transition from the convolutional layers of VGG16 to the fully connected layers that will follow.

#The x variable holds the flattened output, which will serve as input for the next layer.
x = Flatten()(vgg.output)


#adds a Dense layer on top of the flattened output (x). The Dense layer represents a fully connected layer,
#where each neuron is connected to every neuron in the previous layer. 
#The number of neurons in this layer is set to len(folders), which is the number of classes in your dataset

#prediction variable holds the output of this Dense layer, which will be the final prediction of the model.
prediction = Dense(len(folders),activation='softmax')(x)


model = Model(inputs=vgg.input,outputs=prediction)

In [6]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [7]:
#Categorical cross-entropy is commonly used as the loss function for multi-class classification problems, 
#where each input can belong to only one class. It measures the dissimilarity between the predicted class probabilities and the true class labels.

model.compile(
    loss = 'categorical_crossentropy',
    optimizer = 'adam',
    metrics=['accuracy']
)

In [8]:
from keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True
)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('/kaggle/input/potato-disease/train',
                                                 target_size = (224,224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical'
                                                 )

test_set = test_datagen.flow_from_directory('/kaggle/input/potato-disease/test',
                                            target_size = (224,224),
                                            batch_size = 32,
                                            class_mode='categorical'
                                            )

Found 1722 images belonging to 3 classes.
Found 215 images belonging to 3 classes.


In [9]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=50,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/tmp/ipykernel_23/1167563505.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/50
54/54 [==============================] - 78s 896ms/step - loss: 2.7722 - accuracy: 0.4803 - val_loss: 0.8764 - val_accuracy: 0.4791
Epoch 2/50
54/54 [==============================] - 29s 529ms/step - loss: 0.7993 - accuracy: 0.6516 - val_loss: 0.5290 - val_accuracy: 0.8140
Epoch 3/50
54/54 [==============================] - 30s 549ms/step - loss: 0.5938 - accuracy: 0.7863 - val_loss: 0.4959 - val_accuracy: 0.8093
Epoch 4/50
54/54 [==============================] - 30s 544ms/step - loss: 0.4173 - accuracy: 0.8473 - val_loss: 0.4171 - val_accuracy: 0.8512
Epoch 5/50
54/54 [==============================] - 30s 546ms/step - loss: 0.3894 - accuracy: 0.8583 - val_loss: 0.3411 - val_accuracy: 0.8698
Epoch 6/50
54/54 [==============================] - 30s 553ms/step - loss: 0.2561 - accuracy: 0.9042 - val_loss: 0.8552 - val_accuracy: 0.7535
Epoch 7/50
54/54 [==============================] - 30s 544ms/step - loss: 0.2536 - accuracy: 0.9129 - val_loss: 0.3786 - val_accuracy: 0.8465